In [68]:
"""
A améliorer:
* Récupération JSON, cf: https://medium.com/@harshitdy/how-to-get-only-json-response-from-any-llm-using-langchain-ed53bc2df50f
* [ok] Prompt de base
* [ok] Interaction avec requête initiale, distinction "à partir de" ou depuis
"""

import urllib.parse
import json

In [69]:
input = '''{
   "author": "laurent gaudé",
 "start_date": 2008,
 "end_date": null,
 "subject": null
}'''

search = "Un roman de Laurent Gaudé publié en 2008"

# parse json:

input = json.loads(input)

In [70]:
# Fonction: encoder en URL si pas égal à "null"

def encode_url(param):
    if param is not None:
        return urllib.parse.quote(param)

In [71]:
# Fonction: générer une URL Swisscovery UNIGE à partir des paramètres fournis.

def create_swisscovery_url(input):
    base_url = "https://unige.swisscovery.slsp.ch/discovery/search"
    author = input.get("author")
    subject = input.get("subject")
    start_date = input.get("start_date")
    end_date = input.get("end_date")

    # Encoder en URL

    author_encoded = encode_url(author)
    subject_encoded = encode_url(subject)
    start_date_encoded = encode_url(str(start_date))
    end_date_encoded = encode_url(str(end_date))

    # Ajouter les paramètres uniquement s'ils ne sont pas null
    
    query_parts = []
    
    if author is not None:
        author_encoded = encode_url(author)
        query_parts.append(f"query=creator,contains,{author_encoded},AND")

    if subject is not None:
        subject_encoded = encode_url(subject)
        query_parts.append(f"query=any,contains,{subject_encoded},AND")

    # Ajouter la start_date et end_date en corrélation (si l'une est donnée, swisscovery a besoin d'une valeur pour l'autre)
    
    if start_date is not None:
        query_parts.append(f"pfilter=dr_s,exact,{start_date_encoded}0101,AND")
        if end_date is not None:
            query_parts.append(f"pfilter=dr_e,exact,{end_date_encoded}1231,AND")
        else:
            # Distinction entre publié en 2008 et publié depuis 2008
            keywords = ["depuis", "à partir de", "from", "since"]
            if any(keyword in search for keyword in keywords):
                query_parts.append(f"pfilter=dr_e,exact,99991231,AND")
            else:
                query_parts.append(f"pfilter=dr_e,exact,{start_date_encoded}1231,AND")
    else:
        if end_date is not None:
            query_parts.append(f"pfilter=dr_s,exact,-99990101,AND")
            query_parts.append(f"pfilter=dr_e,exact,99991231,AND")

    # Ajouter les paramètres fixes de l'URL
    
    query_parts.extend([
        "tab=41SLSP_UGE_MyInst_CI",
        "search_scope=MyInst_and_CI",
        "vid=41SLSP_UGE:VU1",
        "mode=advanced"
    ])

    url = f"{base_url}?" + "&".join(query_parts)
    return url

In [72]:
# Générer et afficher l'URL

result_url = create_swisscovery_url(input)
print(result_url)

https://unige.swisscovery.slsp.ch/discovery/search?query=creator,contains,laurent%20gaud%C3%A9,AND&pfilter=dr_s,exact,20080101,AND&pfilter=dr_e,exact,20081231,AND&tab=41SLSP_UGE_MyInst_CI&search_scope=MyInst_and_CI&vid=41SLSP_UGE:VU1&mode=advanced
